In [1]:
from sdv import Metadata
from test_20_tables import fetch_data_from_sqlite

metadata, tables = fetch_data_from_sqlite()
metadata = Metadata(metadata)
metadata

Metadata
  root_path: .
  tables: ['Student', 'Professor', 'Course', 'Department', 'Enrollment', 'Assignment', 'Submission', 'Schedule', 'Major', 'Textbook', 'CourseTextbook', 'Library', 'Book', 'BookLoan', 'ResearchGroup', 'ResearchProject', 'ProjectMember', 'Lab', 'LabEquipment', 'EquipmentMaintenance']
  relationships:
    Enrollment.course_id -> Course.course_id
    Enrollment.student_id -> Student.student_id
    Assignment.course_id -> Course.course_id
    Submission.student_id -> Student.student_id
    Submission.assignment_id -> Assignment.assignment_id
    Schedule.course_id -> Course.course_id
    Schedule.professor_id -> Professor.professor_id
    Major.department_id -> Department.department_id
    CourseTextbook.course_id -> Course.course_id
    CourseTextbook.textbook_id -> Textbook.textbook_id
    Book.library_id -> Library.library_id
    BookLoan.student_id -> Student.student_id
    BookLoan.book_id -> Book.book_id
    ResearchProject.group_id -> ResearchGroup.group_id
  

In [2]:
metadata.to_json("1.json")

In [3]:
#from sdv.utils import display_tables

#display_tables(tables)
for name, table in tables.items():
    print(name, table.shape)
from test_20_tables import save_tables
save_tables(tables, "input.xlsx")

Student (10000, 5)
Professor (10000, 4)
Course (10000, 4)
Department (10000, 3)
Enrollment (10000, 5)
Assignment (10000, 4)
Submission (11081, 5)
Schedule (10000, 5)
Major (10000, 3)
Textbook (10000, 4)
CourseTextbook (10000, 2)
Library (10000, 3)
Book (10000, 5)
BookLoan (10000, 5)
ResearchGroup (10000, 3)
ResearchProject (10000, 5)
ProjectMember (10000, 2)
Lab (10000, 3)
LabEquipment (10000, 3)
EquipmentMaintenance (10000, 4)


In [1]:
from sdv.relational import HMA1
import sys
import warnings
warnings.filterwarnings('ignore')
#from tutorials.newhma import NewModel as HMA1
#from tutorials.redirector import WriteableRedirector

#with WriteableRedirector():
model = HMA1(metadata)    

NameError: name 'metadata' is not defined

In [5]:
import logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')# Configure stream handler for the cells
chandler = logging.StreamHandler()
chandler.setLevel(logging.WARNING)
chandler.setFormatter(formatter)
logging.basicConfig(handlers=[logging.FileHandler('sdv.log'), chandler])
logging.log(0, "1223")

In [6]:

model.fit(tables)

Fitting HMA1:   0%|          | 0/20 [00:00<?, ?it/s]

Modeling Student
Modeling Submission



Student:Submission: 100%|██████████| 1/1 [03:50<00:00, 230.22s/it]


Modeling BookLoan



Student:BookLoan: 100%|██████████| 1/1 [00:37<00:00, 37.13s/it]


...


Book:BookLoan: 100%|██████████| 1/1 [00:54<00:00, 54.43s/it]

Fitting HMA1:  60%|██████    | 12/20 [39:18<23:46, 178.32s/it]

Modeling ResearchGroup
Modeling ResearchProject



ResearchProject:ProjectMember: 100%|██████████| 1/1 [00:06<00:00,  6.76s/it]

Fitting HMA1:  75%|███████▌  | 15/20 [45:15<13:03, 156.68s/it]

Modeling Lab
Modeling LabEquipment
Modeling EquipmentMaintenance



LabEquipment:EquipmentMaintenance: 100%|██████████| 1/1 [05:30<00:00, 330.64s/it]

Fitting HMA1: 100%|██████████| 20/20 [52:15<00:00, 156.77s/it]


3s

In [10]:
new_data = model.sample()
new_data

KeyboardInterrupt: 

8s，很快

In [ ]:
save_tables(new_data)

In [ ]:
for name, table in new_data.items():
    print(name, table.shape)

In [9]:
model.save('my_model_10k.pkl')

In [ ]:
from sdv.relational import HMA1
import warnings
warnings.filterwarnings('ignore')
loaded = HMA1.load('my_model_10k.pkl')
new_data = loaded.sample()
new_data.keys()

  0%|          | 0/20 [00:00<?, ?it/s]

Student Childing:   0%|          | 0/3 [00:00<?, ?it/s]

Enrollment:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
model.sample(num_rows=5)

In [ ]:
data_single = model.sample('Schedule', num_rows=100, sample_children=False)

In [ ]:
save_tables({"Schedule":data_single}, "output_single.xlsx")

In [ ]:
#—— 生成Schedule表，并生成关联外键表
data_subset = model.sample('Lab', num_rows=100, sample_children=True)

In [ ]:
save_tables(data_subset, "output_subset.xlsx")